In [ ]:
STACK = "Z0"
EPSILON = "ε"
DELTA = "𝛿"

In [ ]:
import itertools
from collections import defaultdict,deque

In [ ]:
class State:
    name = ""
    transitions = []
    isInitial = False
    isFinal = False

    def __init__(self, name, isInitial, isFinal, transitions):
        self.name = name
        self.isInitial = isInitial
        self.isFinal = isFinal
        self.transitions = transitions

    def addTransition(self, transition):
        self.transitions.add(transition)

    def setFinal (self, isFinal):
        self.isFinal = isFinal

    def setInitial (self, isInitial):
        self.isInitial = isInitial

    def __str__(self):
        string = self.name
        return string

In [ ]:
class Transition:
    inputSymbol = ''
    currState = ''
    nextState = ''
    popSymbol = ''
    pushSymbols = []

    def __init__(self, inputSymbol, currState, nextState, popSymbol, pushSymbols):
        self.inputSymbol = inputSymbol
        self.currState = currState
        self.nextState = nextState
        self.popSymbol = popSymbol
        self.pushSymbols = pushSymbols

    def addPushSymbol(self, sym):
        self.pushSymbols.add(sym)

    def __str__(self):
        pushString = ''.join(self.pushSymbols)
        string = self.inputSymbol + ", " + self.popSymbol + ", " + pushString
        return string


In [ ]:
class IllegalVariableException(Exception):
    def __init__(self, character, rule):
        self.st1 = "Illegal character not in states or terminals"
        self.st2 = character + " in " + rule
    def __str__(self):
        string =  self.st1 + '\n' + self.st2
        return string

In [ ]:
def inputGrammar (data):
    transitions = []
    initStateSym = data[0].rstrip()

    terminals = data[1].rstrip()

    initState = State("Q0", True, False, [])
    midState = State("Q1", False, False, [])
    #finalState = State("Q2", False, True, [])

    terminals = terminals.split(',')
    init = Transition(EPSILON, initState, midState, STACK, [initStateSym, STACK])
    transitions.append(init)
    for idx in range(2, data.__len__()):
        rule = data[idx].strip()
        rule = rule.replace(" ", "")
        for character in rule:
            if character == '-' or character == '>' or character == '|':
                pass
            elif character not in terminals and (not character.isupper()) and character != LAMBDA:
                raise IllegalVariableException(character, rule)

        #left hand side: state
        lhs = rule[:rule.find('-')]
        #right hand side: productions
        rhs = rule[rule.find('>') + 1:]
        #split productions
        rhs = rhs.split('|')
        #assign productions to each state
        for t in rhs:
            trans = Transition(
                EPSILON, #terminal
                midState,
                midState,
                lhs, #pop rule symbol on transition
                list(t)
            )
            transitions.append(trans)
    for t in terminals:
        trans = Transition(
            t, #terminal
            midState,
            midState,
            t, #pop rule symbol on transition
            EPSILON
        )
        transitions.append(trans)

    states = [initState, midState]
    return states, transitions

In [ ]:
class Automaton:
    def __init__(self, states, transitions):
        self.states = states
        self.transitions = transitions

    def toPda(self):
        trFunc = {}
        print("PDA Transitions:")
        for t in self.transitions:
            key = (t.currState, t.inputSymbol, t.popSymbol)

            if (not key in trFunc.keys()):
                trFunc[key] = []
            trFunc[key].append((t.nextState, t.pushSymbols))
        trFunc = enumerate(trFunc.items())
        for i, (key, value) in trFunc:
            print(DELTA + '(' + str(key[0]) + ','
            + key[1] + ',' + key[2] + ') = {', end="")
            targets = []
            for val in value:
                pushStr = ''.join(val[1])
                targets.append(
                    '(' + str(val[0]) + ',' + pushStr + ')'
                )
            print(str(', ').join(targets) + ' }')





In [ ]:
from tabulate import tabulate

class CYK:
    def __init__(self, grammar, startstate):
        self.grammar = grammar
        self.startstate = startstate

    def __getValidCombinations(self, left_collection_set, right_collection_set):
        valid_combinations = []
        for num_collection, left_collection in enumerate(left_collection_set):
            right_collection = right_collection_set[num_collection]
            for left_item in left_collection:
                for right_item in right_collection:
                    combination = left_item + right_item
                    for key, value in self.grammar.items():
                        if combination in value:
                            if not key in valid_combinations:
                                valid_combinations.append(key)
        return valid_combinations

    def __getCollectionSets(self, full_table, x_position, x_offset):
        table_segment = []
        y_position = 0
        while x_offset >= 2:
            item_set = full_table[y_position][x_position:x_position+x_offset]
            if x_offset > len(item_set):
                return None
            table_segment.append(item_set)
            x_offset -= 1
            y_position += 1
        vertical_combinations = []
        horizontal_combinations = []
        for item in table_segment:
            vertical_combinations.append(item[0])
            horizontal_combinations.append(item[-1])
        return vertical_combinations[::-1], horizontal_combinations

    def __generateTable(self, word):
        table = [[]]
        for letter in word:
            valid_states = []
            for key, value in self.grammar.items():
                if letter in value:
                    valid_states.append(key)
            table[0].append(valid_states)
        for x_offset in range(2,len(word)+1):
            table.append([])
            for x_position in range(len(word)):
                collection_sets = self.__getCollectionSets(table, x_position, x_offset)
                if collection_sets:
                    table[-1].append(self.__getValidCombinations(*collection_sets))

        return table

    def checkWord(self, word):
        return self.startstate in self.__generateTable(word)[-1][-1]

    def outputTable(self, word):
        table = self.__generateTable(word)
        pretty_table = [
            [
                ",".join(sorted(y)) if y != [] else u"\u2205" for y in x
            ] for x in table
        ]

        print(tabulate(pretty_table, list(word), tablefmt="grid"))

In [ ]:
def remove_nullable_rules(data):
    rules = defaultdict(list)
    initial_state_sym = data[0].strip()
    terminals = data[1].strip().split(',')


    for idx in range(2, len(data)):
        rule = data[idx].strip()
        lhs, rhs = rule.split('->')
        productions = rhs.split('|')
        for production in productions:
            rules[lhs].append(production.strip())
    dir_nullable = set()


    for non_terminal, productions in rules.items():
        for production in rules[non_terminal]:
            if production == 'ε':
                dir_nullable.add(non_terminal)
    # Step 1: Identify nullable symbols
    nullable = set()
    changed = True
    while changed:
        changed = False
        for non_terminal, productions in rules.items():
            if non_terminal not in nullable:
                for production in productions:
                    # Check if the production is ε or all symbols are nullable
                    if production == EPSILON or all(symbol in nullable for symbol in production):
                        nullable.add(non_terminal)
                        changed = True

    # Step 2: Generate new productions by removing nullable symbols
    new_rules = defaultdict(list)
    for non_terminal, productions in rules.items():
        for production in productions:
            # Include original production
            new_rules[non_terminal].append(production)

            # Generate new productions by removing nullable symbols
            nullable_positions = [i for i, symbol in enumerate(production) if symbol in nullable]

            # Iterate through combinations of nullable positions
            for mask in range(1, len(nullable_positions) + 1):
                for combination in itertools.combinations(nullable_positions, mask):
                    new_production = ''.join([production[i] for i in range(len(production)) if i not in combination])
                    if new_production and new_production not in new_rules[non_terminal]:
                        new_rules[non_terminal].append(new_production)

    # Step 3: Construct the final rules
    final_rules = defaultdict(list)
    for non_terminal, productions in new_rules.items():
        for production in productions:
            if production != EPSILON:
                if len(production)==1 and (production in dir_nullable):
                    if(non_terminal==initial_state_sym ):
                        final_rules[non_terminal].append(EPSILON)
                    else:
                      pass
                else:
                    final_rules[non_terminal].append(production)
            elif production == EPSILON and non_terminal==initial_state_sym:
                final_rules[non_terminal].append(EPSILON)

    # Print the final rules
    print("Modified Grammar after removing nullable rules:")
    for non_terminal, productions in final_rules.items():
        print(f"{non_terminal} -> {'|'.join(productions)}")

    return final_rules

In [ ]:
def remove_unit_productions(rules, initial_state_sym):
    def resolve_unit_productions():

        queue = deque()
        for non_terminal in rules:
            queue.append(non_terminal)

        while queue:
            lhs = queue.popleft()
            productions = rules[lhs]
            new_productions = set()

            for production in productions:
                # Check if the production is a unit production (A -> B)
                if len(production) == 1 and production.isupper():
                    unit_nt = production
                    # Add the productions of the unit non-terminal (B) to lhs (A)
                    for prod in rules[unit_nt]:
                        if prod not in new_productions:
                            new_productions.add(prod)
                            queue.append(lhs)
                else:
                    # Add the non-unit production to the new set
                    new_productions.add(production)


            rules[lhs] = list(new_productions)

    # Resolve unit productions
    resolve_unit_productions()

    # Print the final CFG
    final_rules = defaultdict(list)
    for non_terminal, productions in rules.items():
        for production in productions:
            if len(production) != 1 or production == initial_state_sym or not production.isupper():
                final_rules[non_terminal].append(production)

        # Remove duplicates
        final_rules[non_terminal] = list(set(final_rules[non_terminal]))

    print("Modified Grammar after removing unit productions:")
    for non_terminal, productions in final_rules.items():
        print(f"{non_terminal} -> {'|'.join(productions)}")

    return final_rules


In [ ]:
def process_rhs(rules, initial_state_sym,terminals):

    cnf_rules = defaultdict(list)
    fresh_nt_count = 0

    # def fresh_nt():
    #     nonlocal fresh_nt_count
    #     fresh_nt_count += 1
    #     return f'X{fresh_nt_count}'
    def fresh_nt():
        nonlocal fresh_nt_count
        fresh_nt_count += 1
        apostrophes = "'" * fresh_nt_count
        return f'X{apostrophes}'


    # Step 1: Handle productions with mixed terminals and non-terminals
    for lhs, productions in rules.items():
      for production in productions:
        current_production = []
        if len(production) > 1:
            for symbol in production:
                if symbol in terminals:
                    # Check if the terminal already has a corresponding non-terminal
                    existing_nt = None
                    for nt, prod_list in cnf_rules.items():
                        if len(prod_list) == 1 and prod_list[0] == symbol:
                            existing_nt = nt
                            break

                    if existing_nt:
                        # If the terminal already has a corresponding non-terminal, use it
                        current_production.append(existing_nt)
                    else:
                        # Create a new non-terminal for the terminal
                        new_nt = fresh_nt()
                        cnf_rules[new_nt].append(symbol)
                        current_production.append(new_nt)
                else:
                    current_production.append(symbol)

            # Append the current production to the CNF rules for the current lhs
            cnf_rules[lhs].append(''.join(current_production))
        else:
          cnf_rules[lhs].append(production)


    # Print the final CFG in CNF
    print("Modified Grammar after restricting rhs with only non-terminals or single terminal:")
    for lhs, productions in cnf_rules.items():
        print(f"{lhs} -> {'|'.join(productions)}")

    return cnf_rules


In [ ]:
def convert_productions_length(rules):
    def fresh_nt(non_t):
        nonlocal fresh_nt_count
        fresh_nt_count += 1
        apostrophes = "'" * fresh_nt_count
        return f'{non_t}{apostrophes}'

    def custom_len(s: str) -> int:
        count = 0
        i = 0
        length = len(s)

        while i < length:
            if s[i] == "'":
                while i < length and s[i] == "'":
                    i += 1
            else:
                count += 1
                i += 1

        return count

    def parse_production(production):
        symbols = []
        i = 0
        length = len(production)
        while i < length:
            if production[i] == "'":
                start = i - 1
                while i < length and production[i] == "'":
                    i += 1
                symbols.append(production[start:i])
            elif  i!=length-1 and production[i+1]!="'":
                symbols.append(production[i])
                i += 1
            elif i==length-1:
                symbols.append(production[i])
                i += 1
            else:
                i+=1
        return symbols

    cnf_rules = defaultdict(list)
    fresh_nt_count = 0
    rule_map = {}

    for lhs, productions in rules.items():
        fresh_nt_count = 0
        for production in productions:
            if custom_len(production) <= 2:
                cnf_rules[lhs].append(production)
                continue

            current_symbols = parse_production(production)
            #print(current_symbols)
            while custom_len(''.join(current_symbols)) > 2:
                # Parse production and handle multi-character non-terminals
                first, second = current_symbols[:2]
                rule_key = (first, second)

                if rule_key not in rule_map:
                    new_nt = fresh_nt(lhs)
                    cnf_rules[new_nt].append(first + second)
                    rule_map[rule_key] = new_nt
                else:
                    new_nt = rule_map[rule_key]

                current_symbols = [new_nt] + current_symbols[2:]

            if custom_len(''.join(current_symbols)) == 2:
                cnf_rules[lhs].append(''.join(current_symbols))

    print("Modified grammar after converting to the form A->BC|a:")
    for lhs, productions in cnf_rules.items():
        print(f"{lhs} -> {'|'.join(productions)}")

    return cnf_rules


In [ ]:
def cfgtocnf(data):
    initial_state_sym = data[0].strip()
    terminals = data[1].strip().split(',')
    r1=remove_nullable_rules(data)
    r2=remove_unit_productions(r1, initial_state_sym)
    r3=process_rhs(r2, initial_state_sym,terminals)
    r4=convert_productions_length(r3)
    return r4

In [ ]:
if __name__ == "__main__":
    # print("Enter the CFG input (type 'end' when you are done):")
    # lines = []
    # while True:
    #     line = input().strip()
    #     if line.lower() == 'end':
    #         break
    #     lines.append(line)
    lines=["S", "a,b,ε", "S->aAB|BC", "A->aBA|a", "B->aCC|b", "C->aAB|a"]
    states, transitions = inputGrammar(lines)

    pda = Automaton(states, transitions)
    pda.toPda()

    print(" ")
    print("First Converting cfg to cnf first")
    print(" ")
    cnf=cfgtocnf(lines)

    while(True):
        print("Parse targets? (Y/N)")
        inp = input().strip().capitalize()
        if (inp != "Y"): break
        word = input("String to parse: ").strip()
        cyk = CYK(cnf, lines[0])
        print(cyk.checkWord(word))
        cyk.outputTable(word)


PDA Transitions:
𝛿(Q0,ε,Z0) = {(Q1,SZ0) }
𝛿(Q1,ε,S) = {(Q1,aAB), (Q1,BC) }
𝛿(Q1,ε,A) = {(Q1,aBA), (Q1,a) }
𝛿(Q1,ε,B) = {(Q1,aCC), (Q1,b) }
𝛿(Q1,ε,C) = {(Q1,aAB), (Q1,a) }
𝛿(Q1,a,a) = {(Q1,ε) }
𝛿(Q1,b,b) = {(Q1,ε) }
𝛿(Q1,ε,ε) = {(Q1,ε) }
Parse targets? (Y/N)
Y
String to parse: abbaaa
 
First Converting cfg to cnf first
 
Modified Grammar after removing nullable rules:
S -> aAB|BC
A -> aBA|a
B -> aCC|b
C -> aAB|a
Modified Grammar after removing unit productions:
S -> BC|aAB
A -> a|aBA
B -> aCC|b
C -> a|aAB
Modified Grammar after restricting rhs with only non-terminals or single terminal:
S -> BC|X'AB
X' -> a
A -> a|X'BA
B -> X'CC|b
C -> a|X'AB
Modified grammar after converting to the form A->BC|a:
S -> BC|S'B
S' -> X'A
X' -> a
A -> a|A'A
A' -> X'B
B' -> X'C
B -> B'C|b
C -> a|S'B
False
+--------+-----+-----+--------+--------+--------+
| a      | b   | b   | a      | a      | a      |
+========+=====+=====+========+========+========+
| A,C,X' | B   | B   | A,C,X' | A,C,X' | A,C,X' |
+-----